## Structure-based Knowledge Tracing

This notebook will show you how to train and use the SKT. First, we will show how to get the data (here we use a0910 as the dataset). Then we will show how to train a GKT and perform the parameters persistence. At last, we will show how to load the parameters from the file and evaluate on the test dataset.


The script version could be found in [SKT.py](https://github.com/bigdata-ustc/EduKTM/blob/main/examples/SKT/SKT.py)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](https://github.com/bigdata-ustc/EduKTM/blob/main/examples/SKT/prepare_dataset.ipynb)

In [1]:
from EduKTM.GKT import etl

batch_size = 16
train = etl("../../data/assistment_2009_2010/train.json", batch_size=batch_size)
valid = etl("../../data/assistment_2009_2010/test.json", batch_size=batch_size)
test = etl("../../data/assistment_2009_2010/test.json", batch_size=batch_size)


/data/huangweizhe/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
reading data from ../../data/assistment_2009_2010/train.json: 3025it [00:01, 2397.38it/s]
batchify: 100%|██████████| 327/327 [00:00<00:00, 432.65it/s]
reading data from ../../data/assistment_2009_2010/test.json: 856it [00:00, 3526.76it/s]
/data/huangweizhe/EduKTM/EduKTM/utils/torch/extlib/sampler.py:327: UserWarning: Some buckets are empty and will be removed. Unused bucket keys=[104, 108, 115, 119, 120, 122, 125, 127, 129, 130, 134, 143, 147, 157, 159, 160, 163, 165, 166, 169, 173, 174, 178, 181, 184, 188, 189, 192, 193, 194, 196]
  warnings.warn('Some buckets are empty and will be removed. Unused bucket keys=%s' %
batchify: 100%|██████████| 134/134 [00:00<00:00, 618.56it/s]
reading data from ../../data/assistment_2009_2010/test.js

## Training and Persistence

In [2]:
import logging

logging.getLogger().setLevel(logging.INFO)

In [3]:
from EduKTM import SKT
import torch

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = SKT(ku_num=124, graph_params=[
            ['../../data/assistment_2009_2010/correct_transition_graph.json', True],
            ['../../data/assistment_2009_2010/ctrans_sim.json', False]
        ], hidden_num=5)
model.train(train, valid, epoch=2, device=device)
model.save("skt.params")


Epoch 0: 100%|██████████| 327/327 [05:52<00:00,  1.08s/it]


[Epoch 0] SLMoss: 0.445898


evaluating: 100%|██████████| 134/134 [01:27<00:00,  1.53it/s]


[Epoch 0] auc: 0.594213, accuracy: 0.631270


Epoch 1: 100%|██████████| 327/327 [06:32<00:00,  1.20s/it]


[Epoch 1] SLMoss: 0.433631


evaluating: 100%|██████████| 134/134 [01:26<00:00,  1.56it/s]
INFO:root:save parameters to skt.params


[Epoch 1] auc: 0.624451, accuracy: 0.681644


## Loading and Testing

In [4]:
model.load("skt.params")
auc, accuracy = model.eval(test, device=device)
print("auc: %.6f, accuracy: %.6f" % (auc, accuracy))

INFO:root:load parameters from skt.params
evaluating: 100%|██████████| 134/134 [01:13<00:00,  1.81it/s]

auc: 0.624451, accuracy: 0.681644
